# Criando um novo espaço vetorial baseado no dataset

## Importando os módulos necessários

In [21]:
import pandas as pd
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow as tf

## Carregando modelo já criado em "model-making-script"

In [22]:
searchModel = keras.models.load_model(
    '../model-making-scripts/searchModel.h5', custom_objects={'KerasLayer': hub.KerasLayer})

OSError: No file or directory found at ../model-making-scripts/searchModel.h5

## Carregando o dataset e corrigindo entradas.

In [ ]:
df = pd.read_csv("./data/transcripts.csv")

del df['Unnamed: 0']

df['regulation'][0]

'Os cursos de graduação da Universidade Federal de Campina Grande conferirão graus acadêmicos de nível superior, nas modalidades bacharelado, licenciatura ou outras\nque assegurem o exercício profissional. '

## Importando os dados para vetorização e criação do espaço vetorial

+ Serão importados na forma de Dataframe, usando Pandas
+ Após importação, é feita a concatenação das variáveis "num", "regulation" e "paragraph", pois juntas formarão o texto usado para produzir a representação vetorial do artigo.
+ Os campos são então armazenados na forma de listas, uma para cada coluna do Dataframe

In [ ]:

# Verifica se o campo 'paragraph' é nulo ou vazio
if df['paragraph'].isna().any() or df['paragraph'].str.strip().empty:
    df['Concatenation'] = df['num'].astype(str) + ' ' + df['regulation'].astype(str)
else:
    df['Concatenation'] = df['num'].astype(str) + ' ' + df['regulation'].astype(str) + ' ' + df['paragraph'].astype(str)

# Restante do código
list = df.values.tolist()
num = []
regulation = []
paragraph = []
sentences = []
for i in range(len(list)):
    regulation.append(list[i][0])
    num.append(list[i][1])
    paragraph.append(list[i][2])
    sentences.append(list[i][4])

df.head()


,regulation,num,paragraph,Chapter,Section,Subsection,Concatenation
0,Os cursos de graduação da Universidade Federal...,Art. 1,-,TÍTULO I\nDAS DISPOSIÇÕES GERAIS,-,-,Os cursos de graduação da Universidade Federal...
1,Os cursos de graduação poderão oferecer habili...,Art. 2,-,TÍTULO I\nDAS DISPOSIÇÕES GERAIS,-,-,Os cursos de graduação poderão oferecer habili...
2,Os cursos de graduação terão suas atividades a...,Art. 3,"§ 1º O ano letivo, independente do ano civil, ...",TÍTULO I\nDAS DISPOSIÇÕES GERAIS,-,-,Os cursos de graduação terão suas atividades a...
3,Os cursos de graduação visam à obtenção de qua...,Art. 4,-,TÍTULO II\nDOS CURSOS DE GRADUAÇÃO,-,-,Os cursos de graduação visam à obtenção de qua...
4,A estrutura curricular de cada curso de gradua...,Art. 5,Parágrafo único. A estrutura curricular de cad...,TÍTULO II\nDOS CURSOS DE GRADUAÇÃO,-,-,A estrutura curricular de cada curso de gradua...


## Criando vetores

+ Para criar-se os vetores, o modelo é utilizado para codificar todos os itens da lista sentence
+ Esta lista, criada anteriormente, possui a concatenação do título, autor e transcrição de cada curso, na mesma ordem em que aparecem no dataset.

In [ ]:
vectors = searchModel.predict(sentences)

6/6 [==============================] - 0s 38ms/step


## Criando espaço vetorial
+ Os vetores criados anteriormente devem ser persistidos em JSON
+ O JSON possui diversos campos, cada um contendo um dado relevante acerca de um artigo.
+ Para cada vetor armazenado no campo "data", existe um metadado salvo em cada um dos outros campos no mesmo índice
+ Verificando-se um dado índice em todos os campos, é possível recuperar todas as informações de um dado.

In [ ]:
import json

lists = vectors.tolist()
data = {"data": lists,
        "num": num,
        "regulation": regulation,
        "paragraph": paragraph,
       }
json_str = json.dumps(data)

f = open('./data/data.json', 'w')
f.write(json_str)
f.close()